In [ ]:
import xarray as xr
import pathlib as pl
import pandas as pd
import pywatershed
import os
import dask
import numpy as np

In [ ]:
all_models = ['01473000','05431486','09112500','14015000']# Create a list of all cutouts

In [ ]:
rootdir = pl.Path('../NHM_extractions/20230110_pois_haj/')# Path to location of cutouts

In [ ]:
#var_output_files = ['hru_actet.nc', 'recharge.nc', 'soil_rechr.nc', 'snowcov_area.nc', 'seg_outflow.nc',]#output files of interest

### Working currently from a single cutout directory

In [ ]:
cm = all_models[2] # sets cutout from list
outvardir = rootdir/ cm / 'output'# stes path to location of NHM output folder where output files are.

In [ ]:
# set the file name for the postprocessed model output file that PEST will read
of_name = 'modelobs.dat'# name of file

In [ ]:
########### Useful code
#indat
#df.index.dtypes
#xr.__version__

In [ ]:
#all_nc_file = sorted([i for i in (rootdir/ cm / 'output').glob('model_custom_output.nc')])#Read in the files to check

In [ ]:
# make a file to hold the consolidated results
ofp = open(rootdir / cm / 'modelobs.dat', 'w') # the 'w' will delete any existing file here and recreate; 'a' appends

### Reading in the model_custom_output.nc
#### This file was produced by the old version of the run script that Mike has since updated. The updated script does much more

In [ ]:
cbh = xr.open_dataset(rootdir / cm /'cbh.nc')# Read in combined input data from mode extraction
cbh = cbh.rename({'hruid': 'hru_id'})# rename ot make comapitble with pyWatershed.
#need to split input into separate files for pyWatershed.
cbh[['tmax']].to_netcdf(rootdir / cm /'tmax.nc')
cbh[['tmin']].to_netcdf(rootdir / cm /'tmin.nc')
cbh[['prcp']].to_netcdf(rootdir / cm /'prcp.nc')
cbh.close()

## Need to make the params.json file before preceeding

In [ ]:
# Run model using pyWatershed to get an output file....For now, navigate in a promt to the folder and run the "run_pynhm" file

In [ ]:
#cwd = os.getcwd()
#print(cwd)
#%run ../NHM_extractions/20230110_pois_haj/05431486/run-pynhm.py Ask mike how to do this the right way.

In [ ]:
modelobsdat  = xr.open_dataset(outvardir / 'model_custom_output.nc')

In [ ]:
#modelobsdat

### Slice output to calibration periods for each variable
#### These are as follows from table 2 (Hay and others, 2023):


In [ ]:
aet_start = '2000-01-01'
aet_end = '2010-12-31'
recharge_start = '2000-01-01'
recharge_end = '2009-12-31'
runoff_start = '1982-01-01'
runoff_end = '2010-12-31'
soil_rechr_start = '1982-01-01'
soil_rechr_end = '2010-12-31'
sca_start = '2000-01-01'
sca_end = '2010-12-31'
seg_outflow_start = '2000-01-01'
seg_outflow_end = '2010-12-31'

### Actual ET
#### Get and check the daily data

In [ ]:
#actet_daily = (xr.open_dataset(outvardir / 'hru_actet.nc')['hru_actet']).sel(time=slice(aet_start, aet_end))
actet_daily = modelobsdat.hru_actet.sel(time=slice(aet_start, aet_end))

In [ ]:
#actet_daily

#### Post-process daily output to match observation targets of "monthly" and "mean monthly"

In [ ]:
#Creates a dataframe time series of monthly values (average daily rate for the month)
actet_monthly = actet_daily.resample(time = 'm').mean()

In [ ]:
#actet_monthly

In [ ]:
# Creates a dataframe time series of mean monthly (mean of all jan, feb, mar....)
actet_mean_monthly = actet_monthly.groupby('time.month').mean()

In [ ]:
#actet_mean_monthly

In [ ]:
### Used for plotting/checking
#df = actet_monthly.to_dataframe()# makes a dataframe for plotting and writing out
#df_r = np.ravel(actet_monthly_df, order = 'C')# flattens the 2D array to a 1D array--just playing
#df[8465:8473]
#df.xs(5625, level=1).plot()

#### Now write values to the template file

In [ ]:
inds = [f'{i.year}_{i.month}:{j}' for i in actet_monthly.indexes['time'] for j in actet_monthly['nhm_id'].values]# set up the indices in sequence
varvals = np.ravel(actet_monthly, order = 'C')# flattens the 2D array to a 1D array--just playing 

with open(rootdir / cm / of_name, encoding="utf-8", mode='w') as ofp:
    ofp.write('obsname    obsval\n') # writing a header for the file
    [ofp.write(f'actet_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
actet_monthly.sel(time='2000-01-31').values # look at a slice of the netcdf and compare to pest write

In [ ]:
#[(i,j) for i,j in zip(inds,varvals)] # playing around and learning adv lists making

In [ ]:
inds = [f'{i}:{j}' for i in actet_mean_monthly.indexes['month'] for j in actet_mean_monthly['nhm_id'].values]
varvals =  np.ravel(actet_mean_monthly, order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'actet_mean_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
#actet_mean_monthly

In [ ]:
#actet_mean_monthly.sel(month=1).values # look at a slice of the netcdf and compare to pest write file

### Post Process recharge for calibration use
#### Get daily output file from NHM for recharge

In [ ]:
#recharge_daily = (xr.open_dataset(outvardir / 'recharge.nc')['recharge']).sel(time=slice(recharge_start, recharge_end))
recharge_daily = modelobsdat.recharge.sel(time=slice(recharge_start, recharge_end))

#### Post-process daily output to match observation target of "annual recharge" as an average daily rate for the year

In [ ]:
recharge_annual = recharge_daily.resample(time = 'Y').mean()
recharge_annual_norm = (recharge_annual - recharge_annual.min())/(recharge_annual.max()-recharge_annual.min())

#### Write values to template file

In [ ]:
inds = [f'{i.year}:{j}' for i in recharge_annual_norm.indexes['time'] for j in recharge_annual_norm['nhm_id'].values]
varvals =  np.ravel(recharge_annual_norm, order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'recharge_ann:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
recharge_annual_norm

In [ ]:
#recharge_annual_norm.sel().values # look at a slice of the netcdf and compare to pest write file

### Post Process "soil_rechr" to compare to target
#### Get daily output file from NHM for soil recharge and normalize 0-1

In [ ]:
#soil_rechr_daily = (xr.open_dataset(outvardir / 'soil_rechr.nc')['soil_rechr']).sel(time=slice(soil_rechr_start, soil_rechr_end))
soil_rechr_daily = modelobsdat.soil_rechr.sel(time=slice(soil_rechr_start, soil_rechr_end))

In [ ]:
#Creates a dataframe time series of monthly values (average daily rate for each month)
soil_rechr_monthly = soil_rechr_daily.resample(time = 'm').mean()
soil_rechr_monthly_norm = (soil_rechr_monthly - soil_rechr_monthly.min())/(soil_rechr_monthly.max()-soil_rechr_monthly.min())

#Creates a dataframe time series of annual values (average daily value for each year)
soil_rechr_annual = soil_rechr_daily.resample(time = 'Y').mean()
soil_rechr_annual_norm = (soil_rechr_annual - soil_rechr_annual.min())/(soil_rechr_annual.max()-soil_rechr_annual.min())

In [ ]:
inds = [f'{i.year}_{i.month}:{j}' for i in soil_rechr_monthly_norm.indexes['time'] for j in soil_rechr_monthly_norm['nhm_id'].values]
varvals = np.ravel(soil_rechr_monthly_norm, order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'soil_moist_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
inds = [f'{i.year}:{j}' for i in soil_rechr_annual_norm.indexes['time'] for j in soil_rechr_annual_norm['nhm_id'].values]
varvals =  np.ravel(soil_rechr_annual_norm, order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'soil_moist_ann:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
soil_rechr_annual_norm

In [ ]:
#soil_rechr_monthly_norm.sel(time='1982-01-31').values # look at a slice of the netcdf and compare to pest write file
#soil_rechr_annual_norm.sel(time='1982-12-31').values # look at a slice of the netcdf and compare to pest write file

In [ ]:
#actet_mean_monthly.sel(month=1).values # look at a slice of the netcdf and compare to pest write file

### Post Process "hru_outflow" to compare to target
#### Get and check the daily data

In [ ]:
# These units are in cubic feet (implied per day)
hru_streamflow_out_daily = modelobsdat.hru_streamflow_out.sel(time=slice(runoff_start, runoff_end))

In [ ]:
hru_streamflow_out_monthly = hru_streamflow_out_daily.resample(time = 'm').mean()

In [ ]:
#This converts the average daily rate to a rate in cubic feet per second to compare to observation
hru_streamflow_out_rate = (hru_streamflow_out_monthly)/(24*60*60)

In [ ]:
inds = [f'{i.year}_{i.month}:{j}' for i in hru_streamflow_out_rate.indexes['time'] for j in hru_streamflow_out_rate['nhm_id'].values]
varvals = np.ravel(hru_streamflow_out_rate, order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'runoff_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
#hru_streamflow_out_rate

In [ ]:
#hru_streamflow_out_rate.sel(time='1982-01-31T00:00:00.000000000').values # look at a slice of the netcdf and compare to pest write file

### Post Process "snowcov_area" to compare to target
#### Get and check the daily data

In [ ]:
#snowcov_area_daily = (xr.open_dataset(outvardir / 'snowcov_area.nc')['snowcov_area']).sel(time=slice(sca_start, sca_end))
snowcov_area_daily = modelobsdat.snowcov_area.sel(time=slice(sca_start, sca_end))
remove_ja = True #This is used the filter for removing July and August from the dataset

In [ ]:
#Applying filter to remove months, July and August, from the dataset so same size as obs data.
if remove_ja:
    snowcov_area_daily_restr = snowcov_area_daily.sel(time=snowcov_area_daily.time.dt.month.isin([1, 2, 3, 4, 5, 6, 9, 10, 11, 12]))
else:
    snowcov_area_daily_restr = snowcov_area_daily
snowcov_area_daily.close()     

In [ ]:
inds = [f'{i.year}_{i.month}_{i.day}:{j}' for i in snowcov_area_daily_restr.indexes['time'] for j in snowcov_area_daily_restr['nhm_id'].values]
varvals = np.ravel(snowcov_area_daily_restr, order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'sca_daily:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Get the daily streamflow values from segments associated with the gage pois

In [ ]:
# Get seg_outflow data
#seg_outflow_daily = (xr.open_dataset(outvardir / 'seg_outflow.nc')['seg_outflow']).sel(time=slice(seg_outflow_start, seg_outflow_end),nhm_seg= poi_gage_nhm_seg)
seg_outflow_daily = modelobsdat.seg_outflow.sel(time=slice(seg_outflow_start, seg_outflow_end))

In [ ]:
seg_outflow_daily

In [ ]:
###### Warning: You must run the EFC notebook prior to this block to create the new sf file with EFC codes "EFC_netcdf"
######          First read in the streamflow obs to get the EFC codes; these are needed to write the same obsname as in the obs file.
# streamflow_daily is followed by a suffix: "efc"_"high_low" integers
# efc [1, 2, 3, 4, 5] are ['Large flood', 'Small flood', 'High flow pulse', 'Low flow', 'Extreme low flow']
# high_low [1, 2, 3] are ['Low flow', 'Ascending limb', 'Descending limb']


seg_outflow_start = '2000-01-01'
seg_outflow_end = '2010-12-31'

cdat_efc  = xr.open_dataset(rootdir/ cm / 'sf_data_with_EFC.nc').sel(time=slice(seg_outflow_start, seg_outflow_end))
cdat_efc = cdat_efc.fillna(-9999)
cdat_efc = cdat_efc[['efc', 'high_low']]

In [ ]:
cdat_efc


In [ ]:
inds = [f'_{int(cdat_efc["efc"].sel(poi_id=j, time=i).item())}_{int(cdat_efc["high_low"].sel(poi_id=j, time=i).item())}:{i.year}_{i.month}_{i.day}:{j}' for j in seg_outflow_daily['poi_gages'].values for i in seg_outflow_daily.indexes['time']]

#inds = [f'{i.year}_{i.month}_{i.day}:{j}' for j in seg_outflow_daily['poi_gages'].values for i in seg_outflow_daily.indexes['time']]


varvals = np.ravel(seg_outflow_daily, order = 'F')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'streamflow_daily{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
seg_outflow_daily

In [ ]:
#seg_outflow_daily.sel(npoi_gages='01472198').values # look at a slice of the netcdf and compare to pest write file

In [ ]:
#inds

#### Post-process daily output to match observation targets of "monthly" and "mean monthly"

In [ ]:
#Creates a dataframe time series of monthly values (average daily rate for the month)
seg_outflow_monthly = seg_outflow_daily.resample(time = 'm').mean()

In [ ]:
# Creates a dataframe time series of mean monthly (mean of all jan, feb, mar....)
seg_outflow_mean_monthly = seg_outflow_monthly.groupby('time.month').mean()

In [ ]:
#Now write to the pest obs file
inds = [f'{i.year}_{i.month}:{j}' for j in seg_outflow_monthly['poi_gages'].values for i in seg_outflow_monthly.indexes['time'] ]# set up the indices in sequence
varvals = np.ravel(seg_outflow_monthly, order = 'F')# flattens the 2D array to a 1D array--just playing 

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'streamflow_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
inds = [f'{i}:{j}' for j in seg_outflow_mean_monthly['poi_gages'].values for i in seg_outflow_mean_monthly.indexes['month'] ]
varvals =  np.ravel(seg_outflow_mean_monthly, order = 'F')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'streamflow_mean_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]